Сбор данных
====================

* Полезные ссылки<br>
  * [Web Scraping](https://coderlessons.com/tutorials/python-technologies/izuchite-python-web-scraping/python-web-scraping-kratkoe-rukovodstvo).<br>
  * [Python - Regular Expressions](https://www.tutorialspoint.com/python/python_reg_expressions.htm).<br>
  * [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).<br>
  * [AWS Amazon](https://aws.amazon.com/ru/getting-started/).<br> 

In [1]:
import requests as r
from lxml import html 
import urllib3
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import boto3
from urllib.request import urlopen
import datetime
import random
import sqlite3
import pymysql
import os
import webbrowser
import json
from urllib.request import urlopen

In [2]:
last_update_id = 0

result = r.get(
        'https://api.telegram.org/bot1407505601:AAEbW_Ylkz5R0s2XN6LPSimh3-fAByQoBQw/getUpdates',
        params={'offset': last_update_id + 1})
data = result.json()
for update in data['result']:
    last_update_id = update['update_id']
    chat_id = update['message']['chat']['id']

    send_result = r.get(
        'https://api.telegram.org/bot1407505601:AAEbW_Ylkz5R0s2XN6LPSimh3-fAByQoBQw/sendMessage',
        params={'chat_id': chat_id, 'text': 'Привет от LETPY'}
        )

In [4]:
#Объект ответа в HTTP. Получаем инфу о контенте 
url = "https://authoraditiagarwal.com/wpcontent/uploads/2018/05/MetaSlider_ThinkBig-1080x180.jpg"

r = r.get(url, allow_redirects=True) #запрос для выполнения HTTP-запросов GET для URL
for headers in r.headers: 
    print(headers)
    
#загружаем контент

r = r.get(url) 
with open("ThinkBig.png",'wb') as f:
   f.write(r.content)

print (r.headers.get('content-type'))
print (r.headers.get('Server'))

Date
Server
Expires
Cache-Control
Pragma
Link
Set-Cookie
Keep-Alive
Connection
Transfer-Encoding
Content-Type


AttributeError: 'Response' object has no attribute 'get'

In [5]:
#создаем объект Soup 
soup = BeautifulSoup(r.text, 'lxml')

In [6]:
#запишем захваченные данные в файл CSV с именем dataprocessing.csv в этой папке
f = csv.writer(open(' dataprocessing.csv ','w'))
f.writerow(['Title'])
f.writerow([soup.title.text])

25

In [8]:
#scraping Wikipedia to find out all the countries in Asia.


website_url = r.get('https://en.wikipedia.org/wiki/List_of_Asian_countries_by_area').text
soup = BeautifulSoup(website_url, 'lxml')
My_table = soup.find('table',{'class': 'wikitable sortable'})
links = My_table.find_all("a")
Countries = []
for link in links:
    Countries.append(link.get('title'))

df = pd.DataFrame()
df["Country"] = Countries

df

AttributeError: 'Response' object has no attribute 'get'

https://aws.amazon.com/ru/getting-started/ 

In [ ]:
#соскрести данные с URL
data = requests.get("Enter the URL").text

In [ ]:
# для хранения данных в корзину S3 нам нужно создать клиент S3 
s3 = boto3.client('s3')
bucket_name = "our-content"
#Созбдаем сегмент S3
s3.create_bucket(Bucket = bucket_name, ACL = 'public-read')
s3.put_object(Bucket = bucket_name, Key = '', Body = data, ACL = "public-read")

In [ ]:
#Подключаемся к SQL серверу

conn = pymysql.connect(host='127.0.0.1',user='root', passwd = None, db = 'mysql',
charset = 'utf8', port=None)
cur = conn.cursor()
cur.execute("USE scrap")
random.seed(datetime.datetime.now())
def store(title, content):
   cur.execute('INSERT INTO scrap_pages (title, content) VALUES ''("%s","%s")', (title, content))
   cur.connection.commit()

In [ ]:
#получаем данные из вики

def getLinks(articleUrl):
   html = urlopen('http://en.wikipedia.org'+articleUrl)
   bs = BeautifulSoup(html, 'html.parser')
   title = bs.find('h1').get_text()
   content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
   store(title, content)
   return bs.find('div', {'id':'bodyContent'}).findAll('a',href=re.compile('^(/wiki/)((?!:).)*$'))
links = getLinks('/wiki/Kevin_Bacon')
try:
   while len(links) > 0:
      newArticle = links[random.randint(0, len(links)-1)].attrs['href']
      print(newArticle)
      links = getLinks(newArticle)
        
finally:
   cur.close()
   conn.close()

In [ ]:
#Запускаем ХромДрайвер
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/');
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
driver.quit()

In [ ]:
# Извлечение веб данных о системе пользователя
import user_agents
ua = user_agents.parse(ua)
ua.is_bot
ua.is_mobile
ua.os.family
ua.browser.family

In [ ]:
#robots.txt — это файл, используемый для идентификации частей сайта, которые сканерам разрешено просматривать 
#Sitemap - карты сайта, которые помогают сканерам находить контент без необходимости сканировать каждую страницу